# Tool-Using Helpdesk Agent Demo

This notebook demonstrates a **Tool-Using Agent** pattern. We simulate a helpdesk chatbot that can call different tools:
- **UserDirectoryTool**: checks account status
- **KnowledgeBaseTool**: retrieves relevant support articles
- **TicketingTool**: creates or closes helpdesk tickets

We structure the code into **four sections**:
1. **Tools** (external service stubs)
2. **LLM Adapter** (a mock function that decides tool usage or final answer)
3. **Helpdesk Agent** (the core logic that orchestrates tool calls)
4. **Main Orchestration** (where we simulate a user query and run the agent)

While this example resembles a "mini" ReAct approach, we call it a **Tool-Using Agent** because it focuses on selecting from **multiple tools** rather than the detailed chain-of-thought steps. In practice, you can adapt it to match your exact pattern or naming.

## Section 1: Tools
Here we define three **tool functions**:
- `user_directory_tool(user_id)`: Mocks checking an account status.
- `knowledge_base_tool(query)`: Mocks searching a knowledge base.
- `ticketing_tool(action, description)`: Mocks creating or closing helpdesk tickets.

In a real environment, these might call external APIs or databases.

In [ ]:
# tools.py

def user_directory_tool(user_id):
    """
    Mock function to get user info from some directory or IAM system.
    Returns a dict with account status and basic info.
    """
    # Example mock data: user is locked out.
    return {
        "user_id": user_id,
        "account_status": "locked",
        "last_login": "2025-02-15 10:00 AM"
    }

def knowledge_base_tool(query):
    """
    Mock function to search a knowledge base.
    Returns a short snippet from an 'article' matching the query.
    """
    kb_articles = {
        "email lockout": "To resolve an email lockout, confirm the user's identity, reset the password, and unlock the account via the user portal.",
        "password reset": "Users can reset their password by visiting the reset portal or calling the helpdesk."
    }
    # Simple keyword matching for demonstration
    for keyword, article in kb_articles.items():
        if keyword in query.lower():
            return article
    return "No relevant knowledge base article found for that query."

def ticketing_tool(action, description):
    """
    Mock function to create or close a support ticket in a ticketing system.
    'action' can be 'create' or 'close'.
    """
    if action == "create":
        return f"Ticket created with description: '{description}'."
    elif action == "close":
        return f"Ticket closed with note: '{description}'."
    else:
        return f"Unknown ticket action '{action}'."

## Section 2: LLM Adapter
This **mock function** simulates how an LLM might respond to prompts. It returns either a **TOOL_ACTION** (which tool to call next) or a **FINAL_ANSWER**. In production, replace `call_llm(prompt, temperature)` with real calls to LLM APIs like OpenAI, Anthropic, etc.


In [ ]:
# llm_adapter.py

def call_llm(prompt, temperature=0.2):
    """
    Simulated LLM call. In a production scenario, you'd send 'prompt' to an actual LLM.
    We'll parse the conversation text for keywords and return either a 'TOOL_ACTION' or 'FINAL_ANSWER'.
    """
    lower_prompt = prompt.lower()

    # Check if we've already referenced user_directory_tool output
    if "account status" in lower_prompt or "locked" in lower_prompt:
        # Next step: reference knowledge base
        return "TOOL_ACTION: knowledge_base, query='email lockout'"

    elif "knowledgebase response" in lower_prompt:
        # Then we create a ticket
        return "TOOL_ACTION: ticketing, action='create', description='User locked out of email'"

    elif "ticket created" in lower_prompt:
        # Now provide final answer
        return ("FINAL_ANSWER: The user account was locked. We have reset the password "
                "and unlocked the account. A support ticket was created to confirm resolution.")

    else:
        # First step might be to check the user directory
        return "TOOL_ACTION: user_directory, user_id='U12345'"

## Section 3: Helpdesk Agent
This is where the **tool-using logic** resides. The agent:
1. **Collects** the user query.
2. **Calls** the LLM with the conversation so far.
3. **Parses** the LLM response for `TOOL_ACTION` or `FINAL_ANSWER`.
4. If it’s a tool action, **calls** the relevant tool and appends the observation, then loops again.
5. If it’s a final answer, **concludes**.

We add a maximum loop count (`max_rounds`) to avoid infinite cycles.

In [ ]:
# helpdesk_agent.py

import re
from tools import (
    user_directory_tool,
    knowledge_base_tool,
    ticketing_tool
)
from llm_adapter import call_llm

class HelpdeskToolAgent:
    def __init__(self):
        self.conversation_log = []
        self.max_rounds = 5  # to prevent infinite loops

    def handle_user_issue(self, user_query: str):
        """
        Main loop for a tool-using agent:
        1. Keep prompting the LLM with conversation context.
        2. If the LLM requests a tool, call it and record observation.
        3. If the LLM provides a final answer, stop.
        """
        # 1. Add user query to conversation
        self.conversation_log.append(f"User: {user_query}")

        for _ in range(self.max_rounds):
            # 2. Build prompt from conversation log
            prompt = "\n".join(self.conversation_log) + "\nAgent, decide your next step."
            response = call_llm(prompt)
            self.conversation_log.append(f"Agent: {response}")

            # 3. Check if response indicates a tool action or final answer
            if "TOOL_ACTION:" in response:
                tool_call = self.parse_tool_action(response)
                if tool_call:
                    # 4. Call the specified tool
                    tool_name = tool_call["tool"]
                    kwargs = tool_call["kwargs"]
                    observation = self.call_tool(tool_name, **kwargs)
                    self.conversation_log.append(f"Tool Observation: {observation}")
                else:
                    self.conversation_log.append("Tool Observation: Could not parse tool action.")
                    break

            elif "FINAL_ANSWER:" in response:
                final_answer = response.split("FINAL_ANSWER:")[-1].strip()
                return final_answer

        # If we exit the loop without a FINAL_ANSWER, provide fallback
        return "I’m sorry, but I couldn’t resolve your request at this time."

    def parse_tool_action(self, text):
        """
        Look for lines like:
        TOOL_ACTION: user_directory, user_id='U12345'
        We'll parse it with a simple regex.
        """
        match = re.search(r"TOOL_ACTION:\s*(\w+)\s*,\s*(.*)", text)
        if not match:
            return None
        tool_name = match.group(1)
        params_str = match.group(2)  # e.g. user_id='U12345'

        # Extract key='value' pairs
        kwargs = {}
        pairs = re.findall(r"(\w+)='([^']*)'", params_str)
        for k, v in pairs:
            kwargs[k] = v

        return {"tool": tool_name, "kwargs": kwargs}

    def call_tool(self, tool_name, **kwargs):
        """
        Dispatch to the correct tool function.
        """
        if tool_name == "user_directory":
            return user_directory_tool(kwargs.get("user_id", ""))
        elif tool_name == "knowledge_base":
            return knowledge_base_tool(kwargs.get("query", ""))
        elif tool_name == "ticketing":
            return ticketing_tool(
                action=kwargs.get("action", ""),
                description=kwargs.get("description", "")
            )
        else:
            return f"Unknown tool '{tool_name}'."

## Section 4: Main Orchestration
We simulate a user saying they're locked out of their email. The **HelpdeskToolAgent** will decide which tool(s) to call based on the LLM responses and eventually provide a final answer.


In [ ]:
# main.py

if __name__ == "__main__":
    user_query = "Hi, I'm locked out of my email account. Can you help me?"

    agent = HelpdeskToolAgent()
    final_response = agent.handle_user_issue(user_query)

    print("=== AGENT FINAL RESPONSE ===")
    print(final_response)

## How to Run
1. **Run all cells** in the notebook.
2. The output from the final cell should display the agent’s final answer (e.g., a resolution message).
3. You can inspect the agent’s conversation log or tweak the `user_query` to explore different scenarios.

## Key Points
1. **Multiple Tools**: The agent can invoke different tools based on context (user directory, knowledge base, ticketing system).
2. **LLM-Driven**: The LLM decides when to use each tool by returning a `TOOL_ACTION` directive.
3. **Conversation Loop**: Each new step includes the full conversation history (including tool observations) to guide the next LLM response.
4. **Not Strictly ReAct**: This example is labeled as a **Tool-Using Agent** to emphasize choosing among multiple tools rather than a formal chain-of-thought structure. Still, it's conceptually similar.
5. **Extensibility**: You can add more tools or integrate real APIs. You can also adopt structured output formats (like JSON) for more reliable parsing.

Run the notebook to see how the agent arrives at a final resolution by calling the correct tools in sequence.